<a href="https://colab.research.google.com/github/nferrucho/NPL/blob/main/curso3/ciclo3/2_librerias_ml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://drive.google.com/uc?export=view&id=14reVO1X6LsjqJ3cFgoeHxxddZVGfZn3t" width="100%">

# Librerías para Modelamiento
---

En este notebook veremos algunas librerías típicas para modelamiento en _Python_ y su integración con herramientas como `mlflow`

Comenzamos instalando e importando las librerías necesarias:

In [ ]:
!pip install mlflow

In [ ]:
import mlflow
import os

Adicionalmente, utilizaremos un servidor de `mlflow`:

In [ ]:
command = """
mlflow server \
        --backend-store-uri sqlite:///tracking.db \
        --default-artifact-root file:mlruns \
        -p 5000 &
"""
get_ipython().system_raw(command)

Utilizaremos `ngrok` para acceder al tablero de `mlflow`:

In [ ]:
!pip install pyngrok

Ahora debe agregar su token de `ngrok`:

In [ ]:
token = "2RDTkx1xYfhrYr76sGlFIszN9ur_7UDSixSUBcBdW8UmTNuie" # Agregue el token dentro de las comillas
os.environ["NGROK_TOKEN"] = token

Nos autenticamos en ngrok:

In [ ]:
!ngrok authtoken $NGROK_TOKEN

Ahora, lanzamos la conexión con ngrok:

In [ ]:
from pyngrok import ngrok
ngrok.connect(5000, "http")

Especificamos que MLFlow debe usar el servidor que estamos manejando.

In [ ]:
mlflow.set_tracking_uri("http://localhost:5000")

Vamos a crear un experimento en MLFlow para este conjunto de datos:

In [ ]:
exp_id = mlflow.create_experiment(name="iris", artifact_location="mlruns/")

## **1. Motivación**
---

Actualmente, existen distintas librerías para modelamiento en _Python_, esto se debe a las siguientes razones:

- **Abundancia de tareas**: Existen muchas tareas diferentes en machine learning, como la clasificación, la regresión, la agrupación, la detección de anomalías, etc. Cada tarea requiere un enfoque diferente.
- **Diferentes enfoques y algoritmos**: Hay muchos algoritmos diferentes que se pueden utilizar para resolver una tarea de machine learning, en donde cada uno tiene sus propios fortalezas y debilidades.
- **Flexibilidad**: Algunas librerías están diseñadas para ser flexibles y permiten a los usuarios personalizar y crear sus propios algoritmos y modelos.
- **Mejoras constantes**: La investigación en machine learning es muy activa, y los investigadores y desarrolladores están continuamente publicando nuevos algoritmos y mejoras en los existentes.
- **Diferentes niveles de complejidad**: Algunas librerías están diseñadas para ser fáciles de usar y accesibles para principiantes, mientras que otras están destinadas a expertos y requieren un mayor conocimiento y habilidades.

En cuanto a esto, `mlflow` nos ayuda a estandarizar y versionar modelos de distintas librerías y con distintos lenguajes de programación:

<img src="https://drive.google.com/uc?export=view&id=1B2i0jW-76YrOZY6rG80OC_6zc01v06pA" width="80%">

Para la demostración de las librerías, utilizaremos el conjunto de datos Iris, la cual trata de un conjunto de datos clásico utilizado en el aprendizaje automático y la investigación en inteligencia artificial. Este conjunto de datos contiene información sobre las características de tres tipos diferentes de flores Iris: Iris setosa, Iris versicolor e Iris virginica.

El conjunto de datos incluye información sobre las medidas de cuatro características de las flores: la longitud y el ancho del sépalo y pétalo. Estas características se utilizan para identificar y clasificar los diferentes tipos de flores.

Este conjunto de datos se utiliza a menudo como un problema de clasificación para modelos de aprendizaje automático, y es uno de los conjuntos de datos más utilizados para evaluar la precisión y la eficacia de los algoritmos de aprendizaje automático. También se utiliza para explorar y visualizar patrones y relaciones en los datos y para evaluar la importancia de las características en la identificación y clasificación de las flores.

Cargamos el conjunto de datos:

In [ ]:
from sklearn.datasets import load_iris
data = load_iris()

In [ ]:
features, labels, labels_desc = data["data"], data["target"], data["target_names"]

Podemos ver el número (posición) asignado a cada categoría de flor:

In [ ]:
print(labels_desc)

Vamos a trabajar únicamente con `setosa` y `versicolor`:

In [ ]:
mask = (labels == 1) | (labels == 2)
features = features[mask]
labels = labels[mask] - 1

Veamos ejemplos de modelamiento con este conjunto de datos.

## **2. statsmodels**
---

`statsmodels` es una librería de Python para la estimación y el análisis de modelos estadísticos. Proporciona una amplia gama de modelos y herramientas para la modelación y análisis de datos, incluyendo modelos lineales, modelos de series temporales, modelos de regresión y modelos de análisis de varianza.

<img src="https://drive.google.com/uc?export=view&id=1Z_tWT8gwISX_y7aPMHnl7ZDeHyGOrTpF" width="80%">

`statsmodels` es una librería de análisis de datos muy completa que ofrece una amplia gama de funciones y métodos para la estimación, validación y visualización de modelos estadísticos. Es utilizado por investigadores, analistas de datos y científicos de la computación para realizar análisis estadísticos rigurosos y explorar patrones y tendencias en los datos.

<img src="https://drive.google.com/uc?export=view&id=1Ox6TLvcMATIK_j-8p2qytLjCBDHIA9Bv" width="80%">

Entre sus características se encuentran la capacidad de realizar pruebas estadísticas, diagnósticos de modelos, selección de variables y mucho más. Esta librería es una herramienta importante para cualquier persona interesada en la investigación estadística y el análisis de datos.

Comenzamos instalándola:

In [ ]:
!pip install statsmodels

En este caso, implementaremos un modelo de regresión logística con `statsmodels`:

In [ ]:
import statsmodels.api as sm

Vamos a utilizar el `accuracy` como métrica general de desempeño:

In [ ]:
from sklearn.metrics import accuracy_score

Vamos a definir y a entrenar el modelo como una ejecución de `mlflow`:

In [ ]:
with mlflow.start_run(
        run_name="statsmodels", experiment_id=exp_id
        ):
    model = sm.GLM(
        endog=labels,
        exog=features,
        family=sm.families.Binomial(),
        ) # regresión logística
    results = model.fit() # entrenamiento
    y_pred = (results.predict(features) > .5).astype("int") # predicción continúa
    acc = accuracy_score(labels, y_pred) # evaluación
    mlflow.statsmodels.log_model(results, "model") # registro de modelo
    mlflow.log_metrics({
        "accuracy": acc
        }) # registro de métrica

## **3. scikit-learn**
---

`scikit-learn` es una biblioteca de código abierto para aprendizaje automático en Python. Ofrece una amplia gama de algoritmos y técnicas para el análisis de datos y la construcción de modelos de aprendizaje automático, incluyendo regresión, clasificación, agrupamiento, reducción de dimensionalidad y selección de características.

<img src="https://drive.google.com/uc?export=view&id=1Dsd4f1PTiz-Tois1TbJVQnX_sh0_V_a9" width="80%">

`scikit-learn` es una biblioteca muy accesible y fácil de usar, con una interfaz consistente para todos los algoritmos. Además, proporciona una amplia documentación y una comunidad activa de desarrolladores y usuarios.

La biblioteca se utiliza ampliamente en la industria y en la investigación para la construcción y evaluación de modelos de aprendizaje automático. Es una herramienta esencial para cualquier persona interesada en el análisis de datos y el aprendizaje automático en Python.

Veamos cómo entrenar una máquina de soporte vectorial sobre estos datos y registrar el modelo con `mlflow`:

In [ ]:
from sklearn.svm import SVC

with mlflow.start_run(
        run_name="sklearn", experiment_id=exp_id
        ):
    model = SVC().fit(features, labels) # definición del modelo.
    y_pred = model.predict(features) # predicción
    acc = accuracy_score(labels, y_pred) # evaluación
    mlflow.sklearn.log_model(model, "model") # registro de modelo
    mlflow.log_metrics({
        "accuracy": acc
        })

## **4. xgboost**
---

`xgboost` es una biblioteca de aprendizaje automático de código abierto diseñada para resolver problemas de clasificación y regresión. Se basa en el algoritmo de gradient boosting y proporciona una implementación eficiente y escalable para resolver problemas de aprendizaje automático.

<img src="https://drive.google.com/uc?export=view&id=1D1ZqBu5_BfmIQ_XI-dtuXBqehXd0pQot" width="80%">

El algoritmo `xgboost` es conocido por ser uno de los mejores algoritmos de aprendizaje automático en términos de precisión y rendimiento. Se ha utilizado ampliamente en competiciones de aprendizaje automático en línea como Kaggle.

Además de ser un algoritmo de alta precisión, `xgboost` también es fácil de usar y se integra fácilmente con otros paquetes de _Python_ como `numpy`, `pandas` y `scikit-learn`. Esto lo hace una herramienta popular para investigadores y profesionales de la industria que buscan resolver problemas de aprendizaje automático de manera efectiva y eficiente.

El algoritmo Gradient Boosting es una técnica de aprendizaje automático en la que se combinan varios modelos simples (llamados "boosters") para crear un modelo más complejo y preciso. Cada booster es un modelo que aprende de los errores del modelo anterior y trata de corregirlos.

<img src="https://drive.google.com/uc?export=view&id=1GERxdURZCpSl4zt4xGnZhyJ43KKfwD31" width="80%">

El proceso funciona de la siguiente manera:

1. Se inicializa un modelo básico, por ejemplo, un árbol de decisiones sencillo.
2. Se ajusta el modelo a los datos de entrenamiento y se mide el error.
3. Se entrena un segundo modelo, o booster, para corregir los errores del primer modelo.
4. Se combinan ambos modelos en un modelo más complejo.
5. Se repiten los pasos 2 a 4 para entrenar más boosters y mejorar la precisión del modelo.
6. El proceso continúa hasta que se alcance un cierto nivel de precisión o se agoten los boosters disponibles.

Gradient Boosting es una técnica efectiva para resolver problemas de clasificación y regresión. Se utiliza ampliamente en la industria y en la investigación. Además, XGBoost es una implementación popular y eficiente de Gradient Boosting.

Veamos cómo instalar la librería:

In [ ]:
!pip install xgboost

Importamos el modelo:

In [ ]:
from xgboost import XGBClassifier

El entrenamiento del modelo es equivalente a cómo normalmente se realiza en `sklearn`. Aquí están algunos de los hiperparámetros más importantes en `XGBClassifier`:

- `booster`: Especifica el tipo de modelo base que se utilizará en el ensamblaje de XGBoost. Los valores posibles son `gbtree` (árbol de decisiones) y `dart` (ensamblaje aleatorio).
- `max_depth`: Especifica la profundidad máxima del árbol. Un valor más alto significa que el modelo puede tener ramas más profundas, lo que aumenta la complejidad del modelo.
- `learning_rate`: Especifica la tasa de aprendizaje utilizada para actualizar los pesos en el modelo. Un valor más alto significa que el modelo aprende más rápido, pero también aumenta el riesgo de sobreajuste.
- `n_estimators`: Especifica el número de árboles que se usarán en el ensamblaje. Un valor más alto significa que el modelo será más robusto, pero también aumenta el tiempo de entrenamiento y el tamaño del modelo.
- `gamma`: Especifica una penalización en el crecimiento del árbol. Un valor más alto significa que el modelo es menos propenso a crecer ramas innecesarias.
- `min_child_weight`: Especifica una penalización en el crecimiento de las hojas en el árbol. Un valor más alto significa que el modelo será menos propenso a crear hojas con pocas muestras.
- `subsample`: Especifica la fracción de muestras que se usarán en cada árbol. Un valor más bajo significa que el modelo será más robusto, pero también aumenta el tiempo de entrenamiento.

Estos son solo algunos de los hiperparámetros disponibles en `XGBClassifier`. El ajuste adecuado de ellos puede tener un gran impacto en el rendimiento del modelo. Es importante realizar una búsqueda en grid o validación cruzada para encontrar los valores óptimos para su conjunto de datos.

Vamos a definir el modelo, lo entrenamos y registramos dentro de `mlflow`:

In [ ]:
y_pred = model.predict(features)

In [ ]:
with mlflow.start_run(
        run_name="xgboost", experiment_id=exp_id
        ):

    model = XGBClassifier(
            n_estimators = 20,
            max_depth = 7,
            learning_rate = 1e-4,
            ).fit(features, labels) # entrenamiento
    y_pred = model.predict(features)
    acc = accuracy_score(labels, y_pred) # evaluación
    mlflow.xgboost.log_model(model, "model") # registro de modelo
    mlflow.log_metrics({
        "accuracy": acc
        })

## **5. tensorflow**
---

`tensorflow` es una biblioteca de software de código abierto para aprendizaje automático y cálculo numérico en general. Fue desarrollada por Google Brain Team y se lanzó en 2015. `tensorflow` es utilizado para resolver una amplia variedad de problemas de aprendizaje automático, como clasificación, regresión, detección de objetos y procesamiento de lenguaje natural.

En `tensorflow`, los datos y los cálculos se representan como grafos computacionales, con nodos que representan operaciones matemáticas y arcos que representan arreglos de datos multidimensionales conocidos como tensores. Los usuarios pueden crear y entrenar modelos de aprendizaje automático utilizando estos grafos, mientras que `tensorflow` se encarga de administrar la ejecución de los cálculos en una variedad de plataformas, incluidas CPUs, GPUs y dispositivos móviles.

Además de su flexibilidad y potencia, otro aspecto atractivo de `tensorflow` es su amplia comunidad y documentación, lo que significa que hay muchos recursos disponibles para ayudar a los usuarios a resolver problemas y mejorar sus modelos. En general, `tensorflow` es una herramienta muy valiosa para cualquiera interesado en el aprendizaje automático y el cálculo numérico.

<img src="https://drive.google.com/uc?export=view&id=1j5q1PNb3St7UIfuqKXcxSZ2npZO905jb" width="80%">

Una de las principales utilidades de `tensorflow` es que nos permite implementar redes neuronales profundas (Deep Neural Networks - DNNs) de forma sencilla por medio de `keras`. Las DNNs son una clase de modelos de aprendizaje automático basados en el concepto de las redes neuronales artificiales. A diferencia de las redes neuronales tradicionales, que suelen tener solo una o pocas capas ocultas, las DNNs tienen muchas capas ocultas y, por lo tanto, una gran cantidad de parámetros entrenables. Esto les permite aprender representaciones más complejas y abstractas de los datos, lo que las hace muy efectivas para una amplia variedad de tareas.

<img src="https://drive.google.com/uc?export=view&id=1MEsKrMrLLsqkWPSPnLV0_WaPIw4xvX9w" width="80%">

> **Nota**: el uso en detalle de `tensorflow` lo verá en el módulo 5 de deep learning. En este caso nos enfocaremos en su integración con `mlflow`.

Vamos a definir, entrenar y a registrar el modelo de forma equivalente a los casos anteriores:

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Input, Dropout
from keras.optimizers import Adam
from keras.losses import BinaryCrossentropy

with mlflow.start_run(
        run_name="tensorflow", experiment_id=exp_id
        ):

    model = Sequential([
        Input(shape=(4, )),
        Dense(units=32, activation="relu"),
        Dropout(0.3),
        Dense(units=16, activation="relu"),
        Dropout(0.3),
        Dense(units=1, activation="sigmoid")
        ]) # definición de modelo
    model.compile(
        optimizer=Adam(learning_rate=1e-3),
        loss=BinaryCrossentropy(),
        ) # compilado de modelo
    model.fit(features, labels, epochs=100, batch_size=256) # entrenamiento
    y_pred = (model.predict(features) > .5).astype("int") # predicción continúa
    acc = accuracy_score(labels, y_pred) # evaluación
    mlflow.tensorflow.log_model(model, "model") # registro de modelo
    mlflow.log_metrics({
        "accuracy": acc
        })

## **6. Inferencia**
---

Recuerde que una de las ventajas de `mlflow` es que nos permite utilizar modelos de la misma forma, sin importar la librería con la que fueron entrenados.

Por ejemplo, puede registrar cualquiera de los modelos bajo el nombre `iris_model`:

<img src="https://drive.google.com/uc?export=view&id=1mS53DqJCbA2cjM1F_ujRd9asls5YkEeV" width="80%">

También debe asignar el **staging** a `Production` tal y como se realizó en la unidad pasada. Con esto y de forma independiente al modelo que seleccionará, el siguiente código le permitirá obtener predicciones:

In [ ]:
model_name = 'iris_model'
model_version = 1
model = mlflow.pyfunc.load_model(f"models:/{model_name}/{model_version}")
display(model)

Obtenemos las predicciones:

In [ ]:
y_pred = model.predict(features)
display(y_pred[:5])

## Recursos Adicionales
---

Los siguientes enlaces corresponden a sitios donde encontrará información muy útil para profundizar en los temas vistos en este notebook:

- [MLFlow Models](https://mlflow.org/docs/latest/models.html)
- [statsmodels](https://www.statsmodels.org/stable/index.html)
- [scikit-learn](https://scikit-learn.org/stable/)
- [xgboost](https://xgboost.readthedocs.io/en/stable/)
- [tensorflow](https://www.tensorflow.org/)

## Créditos
---

**Profesor**

- [Jorge E. Camargo, PhD](https://dis.unal.edu.co/~jecamargom/)

**Asistente docente**:

- [Juan S. Lara MSc](https://www.linkedin.com/in/juan-sebastian-lara-ramirez-43570a214/)

**Diseño de imágenes:**
- [Brian Chaparro Cetina](mailto:bchaparro@unal.edu.co).

**Universidad Nacional de Colombia** - *Facultad de Ingeniería*